In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [39]:
# ----------------------------
# CONSTANTS FOR EXPERIMENT TWEAKING
# ----------------------------
TRAIN_FILE = 'zip_train.txt'       # path to training file
TEST_FILE = 'zip_test.txt'         # path to test file
BATCH_SIZE = 64                    # Batch size (affects batch normalization performance)
EPOCHS = 7                        # Number of training epochs
LEARNING_RATE = 0.03             # Learning rate for SGD optimizer
MOMENTUM = 0.90                     # Momentum for SGD optimizer (e.g., 0.5, 0.9, or 0.99)
DROPOUT_RATE = 0.5                 # Dropout rate for fully connected layer (0.0 to 1.0)
INIT_STRATEGY = 'effective'        # Weight initialization strategy: 'slow', 'effective', or 'too_fast'
USE_ENSEMBLE = True               # Set to True to train an ensemble of 3 CNN models

# ----------------------------
# Data Processing
# ----------------------------
def load_data(file_path):
    """
    Load the digit dataset from a text file.
    Each row: <label> <256 feature values>
    Reshape each sample into a 16x16 image.
    """
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            tokens = line.strip().split()
            if len(tokens) != 257:  # 1 label + 256 features
                continue
            label = int(float(tokens[0]))  # class label (0-9)
            features = np.array(tokens[1:], dtype=np.float32)
            image = features.reshape(16, 16)  # 16x16 image
            data.append((image, label))
    return data

class DigitDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, label = self.data[idx]
        if self.transform:
            image = self.transform(image)
        # Add channel dimension: shape becomes (1, 16, 16)
        image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)
        label = torch.tensor(label, dtype=torch.long)
        return image, label

# ----------------------------
# CNN Model Definition
# ----------------------------
class ConvNet(nn.Module):
    def __init__(self, dropout_rate=DROPOUT_RATE):
        """
        The CNN has:
         - Three convolutional layers (with weight sharing over spatial locations)
         - Batch normalization after each conv layer
         - ReLU activations after the first two conv layers
         - Tanh activation after the third conv layer (satisfying the sigmoid/tanh requirement)
         - Adaptive pooling to reduce feature map size
         - One fully connected hidden layer with dropout before the final classification layer
        """
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.tanh = nn.Tanh()  # using tanh here
        
        # Adaptive pooling to get a fixed feature map size (e.g., 4x4)
        self.pool = nn.AdaptiveAvgPool2d((4, 4))
        
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(256, 10)  # 10 output classes

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.tanh(x)

        x = self.pool(x)
        x = x.view(x.size(0), -1)  # flatten
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# ----------------------------
# Parameter Initialization Strategies
# ----------------------------
def initialize_weights(model, strategy=INIT_STRATEGY):
    """
    Initialize weights based on the chosen strategy:
      - 'slow': small weights (std=0.001) → learning very slow.
      - 'effective': Kaiming (He) normal initialization.
      - 'too_fast': high variance weights (std=1.0) → learning too fast.
    """
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            if strategy == 'slow':
                nn.init.normal_(m.weight, mean=0.0, std=0.001)
            elif strategy == 'effective':
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
            elif strategy == 'too_fast':
                nn.init.normal_(m.weight, mean=0.0, std=1.0)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)

# ----------------------------
# Training and Evaluation Functions
# ----------------------------
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    avg_loss = running_loss / total
    accuracy = 100. * correct / total
    print(f"Epoch {epoch}: Train Loss: {avg_loss:.4f}, Train Accuracy: {accuracy:.2f}%")
    return avg_loss, accuracy

def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    avg_loss = test_loss / total
    accuracy = 100. * correct / total
    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%")
    return avg_loss, accuracy

# ----------------------------
# Ensemble Prediction Function
# ----------------------------
def ensemble_predict(models, device, data_loader):
    """
    Given a list of trained models, average their outputs for ensemble prediction.
    """
    all_preds = []
    for model in models:
        model.eval()
    with torch.no_grad():
        for inputs, _ in data_loader:
            inputs = inputs.to(device)
            ensemble_outputs = 0
            for model in models:
                outputs = model(inputs)
                ensemble_outputs += outputs
            avg_outputs = ensemble_outputs / len(models)
            _, predicted = avg_outputs.max(1)
            all_preds.extend(predicted.cpu().numpy())
    return np.array(all_preds)



In [40]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load training and test data
train_data = load_data(TRAIN_FILE)
test_data = load_data(TEST_FILE)
train_dataset = DigitDataset(train_data)
test_dataset = DigitDataset(test_data)

# Create DataLoader with specified batch size (this impacts batch normalization statistics)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# ----------------------------
# Create and initialize the model
# ----------------------------
model = ConvNet(dropout_rate=DROPOUT_RATE).to(device)
initialize_weights(model, strategy=INIT_STRATEGY)

# Loss function and optimizer (using SGD with specified learning rate and momentum)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

# ----------------------------
# Training Loop
# ----------------------------
print("Training single CNN model...")
for epoch in range(1, EPOCHS + 1):
    train(model, device, train_loader, optimizer, criterion, epoch)
    test(model, device, test_loader, criterion)

# ----------------------------
# Ensemble Experiment (if enabled)
# ----------------------------
if USE_ENSEMBLE:
    print("\nTraining ensemble of 3 CNN models...")
    models = []
    for i in range(3):
        print(f"\nTraining model {i+1} of the ensemble:")
        m = ConvNet(dropout_rate=DROPOUT_RATE).to(device)
        initialize_weights(m, strategy=INIT_STRATEGY)
        opt = optim.SGD(m.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
        for epoch in range(1, EPOCHS + 1):
            train(m, device, train_loader, opt, criterion, epoch)
        models.append(m)
    # Ensemble prediction on test set
    ensemble_preds = ensemble_predict(models, device, test_loader)
    true_labels = []
    for _, targets in test_loader:
        true_labels.extend(targets.numpy())
    true_labels = np.array(true_labels)
    ensemble_accuracy = (ensemble_preds == true_labels).mean() * 100
    print(f"\nEnsemble Test Accuracy: {ensemble_accuracy:.2f}%")

Using device: cpu
Training single CNN model...
Epoch 1: Train Loss: 0.2770, Train Accuracy: 91.51%
Test Loss: 0.2339, Test Accuracy: 94.37%
Epoch 2: Train Loss: 0.0708, Train Accuracy: 97.74%
Test Loss: 0.1960, Test Accuracy: 95.02%
Epoch 3: Train Loss: 0.0481, Train Accuracy: 98.44%
Test Loss: 0.1799, Test Accuracy: 95.81%
Epoch 4: Train Loss: 0.0387, Train Accuracy: 98.75%
Test Loss: 0.1755, Test Accuracy: 96.06%
Epoch 5: Train Loss: 0.0293, Train Accuracy: 99.16%
Test Loss: 0.1868, Test Accuracy: 95.91%
Epoch 6: Train Loss: 0.0260, Train Accuracy: 99.30%
Test Loss: 0.1699, Test Accuracy: 96.66%
Epoch 7: Train Loss: 0.0167, Train Accuracy: 99.55%
Test Loss: 0.1873, Test Accuracy: 96.26%

Training ensemble of 3 CNN models...

Training model 1 of the ensemble:
Epoch 1: Train Loss: 0.2563, Train Accuracy: 91.77%
Epoch 2: Train Loss: 0.0877, Train Accuracy: 97.56%
Epoch 3: Train Loss: 0.0541, Train Accuracy: 98.29%
Epoch 4: Train Loss: 0.0423, Train Accuracy: 98.61%
Epoch 5: Train Loss: 